# Burgers' equation

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib as mpl
mpl.rcParams['font.size'] = 8
figsize =(8,4)
mpl.rcParams['figure.figsize'] = figsize
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact
from ipywidgets import widgets, FloatSlider
from utils import riemann_tools
from exact_solvers import burgers

Burgers' equation is one of the simplest examples of a nonlinear conservation law given by

\begin{align} 
q_t + \left(\frac{1}{2}q^2\right)_x = 0.
\label{burgers0}
\end{align}

This is actually the invscid Burgers' equation since Burgers originally studied the viscous equation $q_t + \left(\frac{1}{2}q^2\right)_x = \epsilon q_{xx}$. Rather than modeling a particular physical problem, this equation is the simplest equation that captures some key features of gas dynamics: the nonlinear hyperbolic term and viscosity, and it has been used for developing both theory and numerical methods. In a similar manner, Burgers' inviscid equation, which we will simply refer to as Burgers' equation, will allow us to explore the essentials of the Riemann problem for nonlinear conservation laws.

## Shock formation

The quasi-linear form of Burgers' equation is obtained by expanding the flux term,
\begin{align*}
q_t + qq_x = 0.
\end{align*}

The equation looks very similar to an advection equation with the exception that the advection speed depends on the value $q$. This nonlinearity is essentially the same than in the momentum conservation equation in fluid dynamics, and it will have more complex dynamics than its linear counter part.

One way to think about this equation is to assume $q$ is the height of an ocean wave as it approaches the beach. Assuming the initial condition is a hump of water, the equation tells us that the taller parts of the bump propagate faster than the lower ones. In the next figure, we show the solution at two different times for the initial bump to illustrate this behavior.

![Burgers_bump.](./figures/burgers_bump.png)

Notice that at first $q$ remains univalued for every $x$. However, as time evolves, the wave keeps propagating, and the top part overtakes the bottom. In this case, we obtain a triple-valued solution, which is unphysical.The first time this overtaking happens is refered to as the breaking time, which refers to the point where the waves break on the beach. It is also the point where the conservation law, in its differential form, breaks down. However, to understand what should physically happen at this point, we can solve the viscid Burgers equation while taking vanishing viscosity. What we observe is that a discontinuity in the form of a shock is formed, as shown below   

![Burgers_eqarea.](./figures/burgers_equal_area.png)

The question now is where should we place the discontinuity. As we are talking about conservation, it seems sensitive you want to conserve the amount of $q$, so the disconitnuity should be placed such that the areas $A_1$ and $A_2$ are the same. If we zoom in the region where the shock is generated, we can think of the $q$ profile as an specific initial condition for the Riemann problem, where the left and right constant values satisty $q_l>q_r$. Using the integral form of the conservation law, we can find the shock speed such that conservation is enforced. This simply yields the Rankine Hugoniot condition for the Burgers' eq.

\begin{align*}
s(q_r-q_l) &= \frac{1}{2} (q_r^2 - q_l^2),\\
\Rightarrow \ \ \ \ s &= \frac{1}{2}(q_l + q_r).
\end{align*}

where $s$ is the speed of the shock. The solution with the shock on the right figure shows the physical solution to the Burgers' equation. In the case of Riemann problems where $q_l>q_r$, we know the solution will be a shock propagating with speed $s$. However, we still need to solve the problem when $q_l<q_r$.

## Rarefaction wave

In the previous figures, we observed a shock formed on the right. However, we also observed how the left side of the hump extended and rarefied. The solution in this part is a rarefaction wave, and this is the kind of behavior we will observe when $q_l<q_r$.

## Interactive solution

In [ ]:
def c(q, xi):
    "Characteristic speed."
    return q

def plot_riemann_burgers(q_l,q_r,t):
    states, speeds, reval, wave_types = \
            burgers.exact_riemann_solution(q_l,q_r)
    ax = riemann_tools.plot_riemann(states,speeds,reval,
                                    wave_types,t=t,
                                    t_pointer=0,extra_axes=False,
                                    variable_names=['q']);
    riemann_tools.plot_characteristics(reval,c,axes=ax[0])
    ax[1].set_ylim(-0.05,1.05)
    plt.show()    

In [ ]:
interact(plot_riemann_burgers,
         q_l=FloatSlider(min=0.,max=1.,value=0.5,
                           description=r'$q_l$'),
         q_r=FloatSlider(min=0.,max=1.,description=r'$q_r$'),
         t=FloatSlider(min=0.,max=1.,value=0.1));

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(10,7))
x = np.linspace(-13,13,150)
y = np.exp(-0.1*x**2)
plt.plot(x,y,'--k', lw=1)
plt.xlim([-7.5,12.5])
plt.show()